In [1]:
import numpy as np
import sys

sys.path.append("../../src")
from si_explainer import SiExplainer
sys.path.append("../../experiments/1. archdetect")
from synthetic_utils import *

%load_ext autoreload
%autoreload 2

## Parameters

In [2]:
function_id = 1

p = 40 # num features
input_value, base_value = 1, -1

## Get Data and Synthetic Function

In [3]:
input = np.array([input_value]*p)
baseline = np.array([base_value]*p)

print("function id:", function_id)
model = synth_model(function_id, input_value, base_value)
gts = model.get_gts(p)

function id: 1


## Get Explanation

In [4]:
si_method = SiExplainer(model, input=input, baseline=baseline, output_indices=0, batch_size=20)

### Individual Tests

In [5]:
num_T = 1

inters = dict()
for i in range(p):
    for j in range(i+1, p):
        if i == j: continue
        S = (i,j)
        
        att = si_method.attribution(S, num_T)
        inters[S] = att**2
                
for i in range(p):
    att = si_method.attribution([i], num_T)**2

In [6]:
print("auc", get_auc(inters.items(), gts))

auc 1.0


### Batch Version

In [7]:
mat = si_method.batch_attribution(num_T, pairwise=True)
arr = si_method.batch_attribution(num_T, main_effects=True, pairwise=False)

inters = {}
for i in range(p):
    for j in range(i+1, p):
        inters[(i,j)] = mat[i, j]**2

In [8]:
print("auc", get_auc(inters.items(), gts))

auc 1.0
